### Neural Machine Translation

In this notebook we are going to create a neural machine translation model using deep learning with tensoflow.


First we need to import required packages for us to create this model.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf


import os
import string
from string import digits
import matplotlib.pyplot as plt

import re
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow import keras
from google.colab import drive

%matplotlib inline

The data that we will be using will be loaded from a a file that was uploaded in a google drive. We got the data from [kaggle](https://www.kaggle.com/datasets/aiswaryaramachandran/hindienglish-corpora)

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


Then we are going to define the path where our datafile is located.

In [3]:
data_path = '/content/drive/My Drive/NLP Data/Hindi_English_Truncated_Corpus/Hindi_English_Truncated_Corpus.csv'
assert os.path.exists(data_path), f"The file path {data_path} does not exists."

Next we are going to read this file in a pandas dataframe.

In [4]:
df = pd.read_csv(data_path, encoding='utf-8')

Checking the first `5` examples in the dataframe.

In [5]:
df.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


Checking the distribution of the `source` where the translation was comming from.

In [6]:
df['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

We are going to take the english and hindi sentence pairs that are comming from the `ted` source.

In [7]:
df=df[df['source']=='ted']
df.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है


Checking if we have any null values and drop the null values and duplicated.

In [8]:
df.isnull().sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [9]:
df=df[~pd.isnull(df['english_sentence'])]
df.drop_duplicates(inplace=True)

We are going to select the random `25, 000` rows from the dataset, these are the examples that we are going to work with.

In [10]:
df=df.sample(n=25000,random_state=42)
df.shape

(25000, 3)

Next we are going to convert the `target` (hindi) and the `source`(english) sentences to lower case.

In [11]:
df['english_sentence']=df['english_sentence'].apply(lambda x: x.lower())
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.lower())

Then we are going to remove `qoutes` as a method of data cleaning.

In [12]:
# Remove quotes
df['english_sentence']=df['english_sentence'].apply(lambda x: re.sub("'", '', x))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

We are then going to remove special characters


In [13]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
df['english_sentence']=df['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

We are then going to remove numbers from both the target and sorce sentence. We are going to further on remove extra spaces that will be in our target and source sentences.

In [14]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
df['english_sentence']=df['english_sentence'].apply(lambda x: x.translate(remove_digits))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
df['english_sentence']=df['english_sentence'].apply(lambda x: x.strip())
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.strip())
df['english_sentence']=df['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

For the target we are going to add the `START_ ` token at the start of each sentence and ` _END` token at the end of each sentence.

In [15]:
# Add start and end tokens to target sequences
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

Checking the cleaned src and trg sentences.

In [16]:
df.head()

,source,english_sentence,hindi_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापि...
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END
122330,ted,and its not as hard as you think integrate cli...,START_ और जितना आपको लगता है यह उतना कठिन नहीं...


### Vocabulary

In this section we are going to have a look at creating the vocabulary for our `src` and `trg` sentences.

In [17]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in df['english_sentence']:
  for word in eng.split():
    if word not in all_eng_words:
      all_eng_words.add(word)

all_hindi_words=set()
for hin in df['hindi_sentence']:
  for word in hin.split():
    if word not in all_hindi_words:
      all_hindi_words.add(word)

Checking the vocabulary size or number of unique words in the source vocabulary.

In [18]:
len(all_eng_words)

14030

Checking the vocabulary size on the size on the targe.

In [19]:
len(all_hindi_words)

17540

We can create a column that which contains the sentence length for both the source and target as follows:


In [20]:
df['length_eng_sentence']=df['english_sentence'].apply(lambda x:len(x.split(" ")))
df['length_hin_sentence']=df['hindi_sentence'].apply(lambda x:len(x.split(" ")))

Checking the dataframe head with sentences length.

In [21]:
df.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापि...,11,16
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END,2,5
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END,7,8
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END,4,6
122330,ted,and its not as hard as you think integrate cli...,START_ और जितना आपको लगता है यह उतना कठिन नहीं...,16,20


In [22]:
df[df['length_eng_sentence']>30].shape

(0, 5)

Let's filter our data and trim it so that we take the rows that has at most `20` words in a sequence.

In [23]:
df=df[df['length_eng_sentence']<=20]
df=df[df['length_hin_sentence']<=20]
df.shape

(24774, 5)

Checking the sentence in the target and source that has the maximum length.

In [24]:
print("maximum length of Hindi Sentence ",max(df['length_hin_sentence']))
print("maximum length of English Sentence ",max(df['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


Next we are going to get the maximum sentence length for both the src and target sentences.

In [25]:
max_length_src=max(df['length_hin_sentence'])
max_length_tar=max(df['length_eng_sentence'])

Let's then sort our targets and source tokens and get the how many tokens are we having for each of the pairs.

In [26]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(14030, 17540)

We are going to add `1` token for our target sentences as `0` will be the value of the padding token.

In [27]:
num_decoder_tokens += 1 #for zero padding

Creating a `stoi` string to integer mapping for both the source and the target sencences in the corpus.

In [28]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

Then we are going to create an `itos` an integer to string mapping for both the source and target pairs in the corpus.

In [29]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [30]:
df = shuffle(df)
df.head(10)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
117802,ted,i think the market can help us figure that out,START_ मुझे लगता है कि बाज़ार हमें इस का जवाब ...,10,14
18220,ted,it begins about years ago,START_ इसकी शुरुआत लगभग साल पहले हुई _END,5,8
20115,ted,not only do we create much more dignity for them,START_ न सिर्फ़ हम उनके लिये ही स्वाभिमान पैदा...,10,11
84789,ted,theyre not just sitting there,START_ वे वहां आराम से बैठे हुए नहीं हैं _END,5,10
46160,ted,in a decentralized way,START_ विकेन्द्रीकृत तरीके से भी _END,4,6
75011,ted,this simulation was run on processors for a month,START_ इस प्रारूपण को कम्प्युटर प्रोसेसरों पर ...,9,13
73408,ted,one because the benefits of the marshall plan ...,START_ पहले तो इसलिए कि मार्शल प्लान के फायदों...,11,15
112584,ted,if you want to see the features of this car yo...,START_ अगर आप इस कार की विशेषताओं को देखना चाह...,15,18
48424,ted,help people to vote on them,START_ लोगों से उन विचारों पर राय ले रहा था _END,6,11
109504,ted,is the most important thing for development,START_ सबसे महत्वपूर्ण है विकास के लिये _END,7,8


Next we are going to get the source and the target sentences as numpy arrays and split these examle pairs into train and test set.

In [31]:
X, y = df['english_sentence'], df['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((19819,), (4955,))

We can save the `pickle` files for our source and target.

In [32]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

### colate_function

We are then going to create a colate_function called `generate_batch` that will help us to generate batches during training of the model.

In [33]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

We are going to create a  decoder and encoder achitecture. We are going to use the keras functional api to create the encoder and decoder model.

In [34]:
latent_dim=300

#### Encoder Model

In the following code cell we are going to create our `encoder_model`

In [35]:
# Encoder
encoder_inputs = keras.layers.Input(shape=(None,))
enc_emb =  keras.layers.Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

#### Decoder Model

In the following code cell we are going to create our `decoder_model`

In [36]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.layers.Input(shape=(None,))
dec_emb_layer = keras.layers.Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

Compiling the model.

In [37]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

Printing the model summary...

In [38]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    4209000     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    5262300     ['input_2[0][0]']                
                                                                                              

### Trainhing loop

During calling of the `model.fit` we are going to use the `generator` which is the `generate_batch` with the batch size of `128` and train the model for `100` epochs.

In [39]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [40]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

<ipython-input-40-b22c3ec5e69e>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/100
154/154 [==============================] - 79s 422ms/step - loss: 6.9349 - val_loss: 6.3637
Epoch 2/100
154/154 [==============================] - 51s 325ms/step - loss: 6.3067 - val_loss: 6.3228
Epoch 3/100
154/154 [==============================] - 50s 324ms/step - loss: 6.2603 - val_loss: 6.3043
Epoch 4/100
154/154 [==============================] - 51s 329ms/step - loss: 6.2225 - val_loss: 6.2526
Epoch 5/100
154/154 [==============================] - 51s 331ms/step - loss: 6.1390 - val_loss: 6.1691
Epoch 6/100
154/154 [==============================] - 51s 329ms/step - loss: 6.0447 - val_loss: 6.0898
Epoch 7/100
154/154 [==============================] - 50s 323ms/step - loss: 5.9548 - val_loss: 6.0127
Epoch 8/100
154/154 [==============================] - 50s 326ms/step - loss: 5.8791 - val_loss: 5.9615
Epoch 9/100
154/154 [==============================] - 51s 335ms/step - loss: 5.8185 - val_loss: 5.9187
Epoch 10/100
154/154 [==============================] - 50s 325m

Saving the model weights...

In [41]:
model.save_weights('nmt_weights.h5')

In [42]:
# Encode the input sequence to get the "thought vectors"
encoder_model = keras.models.Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = keras.layers.Input(shape=(latent_dim,))
decoder_state_input_c = keras.layers.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = keras.models.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [43]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

Making predictions.

In [44]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [45]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 24ms/step
Input English sentence: human rights is also important but it just gets one cross
Actual Hindi Translation:  मानवाधिकार भी ज़रूरी है मगर उसे थोडे कम नंबर मिले हैं। 
Predicted Hindi Translation:  मानवता को सामाजिक समस्या को देख सकते हैं कि यह सब


In [46]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 21ms/step
Input English sentence: mt in that respect
Actual Hindi Translation:  इसलिए 
Predicted Hindi Translation:  तो ये है कि 


In [47]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 19ms/step
Input English sentence: a priest
Actual Hindi Translation:  एक पुजारी 
Predicted Hindi Translation:  एक दिन 


In [48]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 23ms/step
Input English sentence: we need to open our eyes and see that we are not victims
Actual Hindi Translation:  हमें हमारी आँखे खोलनी होगी और देखना होगा कि हम पीड़ित नहीं हैं 
Predicted Hindi Translation:  हमें पता है कि हम में से अधिक लोग इस बात से नहीं च


### Refs

1. [www.kaggle.com](https://www.kaggle.com/code/aiswaryaramachandran/english-to-hindi-neural-machine-translation/notebook)